In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import copy as cp
import math

### Create TF-IDF matrix


https://www.geeksforgeeks.org/tf-idf-model-for-page-ranking/

1. Create a matrix of TF, term frequency (this is same as bag of words) --->   "binaryIngredientsMat"
2. Normalize the TF matrix ---> "binaryIngredientsMatNorm"
3. Create IDF for each ingredient ---> "idf"
4. Multiple TF * IDF to obtain TF-IDF matrix ---> "tf-idf"

The values in " " are the variable names in this script

### 1. Load Data + Create TF
Here, TF = term frequency. This is the same as Bag of Words, or Binary Matrix generated from Mingyu's BMF script: 

In [124]:
data = pd.read_json('train.json')

In [125]:
# get the list of ingredients
ingredientList = sorted(list(set([ingredient for i in range(data.shape[0]) for ingredient in data.loc[i,'ingredients']])))
# the mapping between ingredient and its index
ingredient2index = dict(zip(ingredientList, range(len(ingredientList))))
# create a binary matrix indicating whether or not an ingredient is in a recipe
binaryIngredientsMat = np.zeros((data.shape[0], len(ingredientList)))
for iRecipe in range(data.shape[0]):
    binaryIngredientsMat[iRecipe, [ingredient2index[ingredient] for ingredient in data.loc[iRecipe, 'ingredients']]] = 1
    
dataBinaryIngredients = pd.DataFrame(binaryIngredientsMat, columns=ingredientList)
dataBinaryIngredients.head()

,( oz.) tomato sauce,( oz.) tomato paste,(10 oz.) frozen chopped spinach,"(10 oz.) frozen chopped spinach, thawed and squeezed dry",(14 oz.) sweetened condensed milk,(14.5 oz.) diced tomatoes,(15 oz.) refried beans,1% low-fat buttermilk,1% low-fat chocolate milk,1% low-fat cottage cheese,1% low-fat milk,"2 1/2 to 3 lb. chicken, cut into serving pieces",2% low fat cheddar chees,2% low-fat cottage cheese,2% lowfat greek yogurt,2% milk shredded mozzarella cheese,2% reduced-fat milk,25% less sodium chicken broth,33% less sodium cooked deli ham,33% less sodium cooked ham,33% less sodium ham,33% less sodium smoked fully cooked ham,40% less sodium taco seasoning,40% less sodium taco seasoning mix,7 Up,"8 ounc ziti pasta, cook and drain",95% lean ground beef,A Taste of Thai Rice Noodles,Accent Seasoning,Adobo All Purpose Seasoning,Alaskan king crab legs,Alexia Waffle Fries,Alfredo sauce,Amarena cherries,Amaretti Cookies,American cheese,Anaheim chile,Angostura bitters,Argo Corn Starch,Asian chili sauce,Asian sweet chili sauce,Azteca Flour Tortillas,BACARDI® Mixers Margarita Mix,BACARDI® Superior,BREAKSTONE'S Sour Cream,Baileys Irish Cream Liqueur,Balsamico Bianco,Barilla Linguine,Barilla Oven-Ready Lasagne,Barilla Plus Pasta,Bartlett Pear,Belgian endive,Bengali 5 Spice,Bertolli Garlic Alfredo Sauce,Bertolli Tomato & Basil Sauce,Bertolli® Alfredo Sauce,Bertolli® Arrabbiata Sauce,Bertolli® Classico Olive Oil,Best Food's Mayonnaise with Lime Juice,Best Foods® Real Mayonnaise,Better Than Bouillon Chicken Base,Betty Crocker™ oatmeal cookie mix,Biryani Masala,Bisquick Baking Mix,Bisquick Original All-Purpose Baking Mix,Bob Evans Italian Sausage,Bordelaise sauce,Boston lettuce,Boursin,Braeburn Apple,Bragg Liquid Aminos,Breakstone’s Sour Cream,Breyers® Natural Vanilla Ice Cream,Budweiser,Burgundy wine,CURRY GUY Smoked Garam Masala,CURRY GUY Smoked Spicy Salt,California bay leaves,Camellia Red Kidney Beans,Campbell's Condensed Cheddar Cheese Soup,Campbell's Condensed Cream of Chicken Soup,Campbell's Condensed Cream of Mushroom Soup,Campbell's Condensed Tomato Soup,Cara Cara orange,Castelvetrano olives,Cavenders Greek Seasoning,Challenge Butter,Chambord Liqueur,Chartreuse Liqueur,Chianti,Chinese egg noodles,Chinese rice vinegar,Chinese rose wine,Chinese sesame paste,Cholula Hot Sauce,Ciabatta rolls,Cinnamon Toast Crunch Cereal,Classico Pasta Sauce,Cointreau Liqueur,Colman's Mustard Powder,...,whole wheat tortilla wraps,whole wheat tortillas,whole wheat uncooked lasagna noodles,wholemeal flour,wide egg noodles,wide rice noodles,wieners,wild asparagus,wild garlic,wild mushrooms,wild rice,wild salmon,wildflower honey,wine,wine syrup,wine vinegar,winesap,wing sauce,winter melon,winter savory,winter squash,wish bone guacamol ranch dress,wish bone ranch dress,wish bone red wine vinaigrett dress,wish-bone,wish-bone light asian sesame ginger vinaigrette dressing,wish-bone light country italian dressing,won ton skins,won ton wrappers,wondra,wondra flour,wonton noodles,wonton skins,wonton wrappers,wood ear mushrooms,wood mushrooms,worcestershire sauce,worcestershire sauce low sodium,xanthan gum,xuxu,yaki-nori,yam bean,yam noodles,yams,yardlong beans,yeast,yeast extract,yellow bean sauce,yellow bell pepper,yellow cake mix,yellow chives,yellow corn,yellow corn meal,yellow crookneck squash,yellow curry paste,yellow food coloring,yellow heirloom tomatoes,yellow hominy,yellow lentils,yellow miso,yellow mustard,yellow mustard seeds,yellow onion,yellow peas,yellow peppers,yellow rice,yellow rock sugar,yellow split peas,yellow squash,yellow summer squash,yellow tomato,yellowfin,yellowfin tuna,yellowtail,yellowtail snapper fillets,yoghurt,yoghurt natural low fat,yogurt cheese,yogurt dressing,yogurt low fat,yolk,yoplait,young coconut meat,young leeks,young nettle,yu choy,yuca,yucca,yucca root,yukon gold,yukon gold potatoes,yuzu,yuzu juice,za'atar,zest,zesty italian dressing,zinfandel,ziti,zucchini,zucchini blossoms
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### 2. Normalize TF Matrix
* For each ingredient in each recipe, divide by the total number of ingredients in that recipe. (Normalize by total number of 'words' in 'document')

* Another way to think of this: for each row, for each ingredient, divide by the total number of ingredients for that row.

In [126]:
%%time

n_ingredients = dataBinaryIngredients.shape[1]
n_recipes = dataBinaryIngredients.shape[0]

binaryIngredientsMatNorm = cp.deepcopy(binaryIngredientsMat)

for row in range(0,n_recipes):
    row_sum = binaryIngredientsMat[row,:].sum()
    binaryIngredientsMatNorm[row,:] = binaryIngredientsMat[row,:]/row_sum
    # Check progress: 
    #if row % 1000 == 0:
    #    print(row, row_sum, binaryIngredientsMatNorm[row,:].sum())

CPU times: user 1.63 s, sys: 911 ms, total: 2.54 s
Wall time: 1.17 s


### 3. Create IDF
Inverse Document Frequency is a measure of how common that term (or ingredient) is in all documents. 

IDF is a vector of ingredients (not a matrix!)

IDF(ingredient) = log( N / df(t))
Where: 
* N = total number of recipes
* df(t) = total number of recipes that contain the ingredient
* We take the log so that it's a monotonically increasing function

High IDF score means the ingredient is less frequently used in all recipes, and thus might have more value.

Low IDF score means the ingredient is more frequently used in all recipes, and thus might have less value. 

In [127]:
# idf(t) = log( N / df(t) )
# idf(ingredient) = N_recipes/ N_recipes_with_that_ingredient

# Each ingredient will have an IDF
idf = np.zeros(len(ingredientList))
for ingredient in range(0,n_ingredients):
    n_recipes_with_that_ingredient = binaryIngredientsMat[:,ingredient].sum()
    idf[ingredient] = math.log( n_recipes / n_recipes_with_that_ingredient )
    # if ingredient % 1000 == 0:
    #    print(idf[ingredient], n_recipes_with_that_ingredient)
print(idf)        

[8.19307344 8.39374413 9.49235642 ... 7.22367288 3.79750258 9.89782153]


### 4. Create TF-IDF matrix
For each ingredient column, multiply the TF * IDF(ingredient): 

binaryIngredientMatNorm[:,ingredient] * idf[ingredient]

In [128]:
# Calculate TF-IDF for every term:

tf_idf_mat = cp.deepcopy(binaryIngredientsMatNorm)

for ingredient in range(0,n_ingredients):
    tf_idf_mat[:,ingredient] = binaryIngredientsMatNorm[:,ingredient]*idf[ingredient]
    #if ingredient % 1000 == 0:
    #    print(tf_idf[:,ingredient])
        

In [129]:
# Make it into a panda dataframe, and save: 
import pickle
pd.options.display.max_columns=200
tf_idf = pd.DataFrame(tf_idf_mat, columns=ingredientList)
tf_idf.to_pickle("tf_idf.pkl")

In [130]:
# Check results: For the ingredient 'salt', recipes that have more ingredients should have a lower final tf-idf value than recipes that have less ingredients.  We see this is indeed the case: recipe 3 only has 5 ingredients, so the value of 'salt' is higher at .19. In comparison, recipe 1,2,4 have way more ingredients, so the value of the ingredient 'salt' is much lower at .03-.07: 
tf_idf['salt'].head()

0    0.000000
1    0.071834
2    0.065848
3    0.197545
4    0.039509
Name: salt, dtype: float64

In [131]:
data['ingredients'].head()

0    [romaine lettuce, black olives, grape tomatoes...
1    [plain flour, ground pepper, salt, tomatoes, g...
2    [eggs, pepper, salt, mayonaise, cooking oil, g...
3                  [water, vegetable oil, wheat, salt]
4    [black pepper, shallots, cornflour, cayenne pe...
Name: ingredients, dtype: object